# 交差検証

## 交差検証とは

サンプルデータをいくつかのグループに分割し、その一部のグループを使ってモデル構築を行い、残りのグループを使ってモデルの良さを評価する手法。また、モデル構築に使用するグループと、モデルの良さを評価するグループを変更して再度評価していく手法。

検証の手順としては以下の通り。
1. サンプルデータを複数のグループに分割する。(分割した数をKとする)
2. いくつかのグループで学習を行う。
3. 残りのグループで検証を行う。
4. 学習するグループと検証を行うグループを変更して再度2.3の手順を踏む。
5. 複数回検証を実施したら、その平均値などから良いモデルを評価する。

イメージは1.2の図参照。

## ホールドアウトとの違い

ホールドアウトがすべてのデータを二つに分けるのに対し、交差検証はいくつかのグループに分けて検証を行う。（詳しくは以下の図を参照）

![ホールドアウト](https://newtechnologylifestyle.net/wp-content/uploads/2018/05/1.Hold-out.png)
![交差検証](https://newtechnologylifestyle.net/wp-content/uploads/2018/05/2.Cross-Validation.png)

## メリット・デメリット

- メリット
  - グループで分割した数(K)だけ、検証やテストを行うことができるため、サンプルのデータ数がある程度少ない場合でも検証を行う事ができる。

- デメリット
  - グループで分割した数(K)だけ、計算コストが多くかかるため、サンプルデータが多い場合などには考えて使用しなければならない。

## 使用例

In [81]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings('ignore')

%matplotlib inline

# データセットを読み込む
iris = load_iris()

# 交差検証の準備：n_splits=分割数、shuffle=シャッフルするかどうか、random_state:None:バラバラ、それ以外:再現性を保つ
k_fold_cross_validation = KFold(n_splits=4, shuffle=True, random_state=1) 

# 山本備忘録： 
# shuffle: 用意されているデータのクラスが順番になってしまっている時はTrueを使うと順番をバラバラにしてからK-Foldを実施する。
#          stratifiedとは違う。iris_data_setの場合、FalseにするとKFoldの場所によってはモデルの識別精度が悪くなっていることがわかる。

# 交差検証の実施
# 今回はn_splitsが4回なので、4回繰り返される。毎度のループごとに訓練データとテストデータに使用するインデックスが渡されている。
for train_index, test_index in k_fold_cross_validation.split(iris.data):

    # 選択されたインデックスをもとにデータを分割
    X_train, y_train = iris.data[train_index], iris.target[train_index]
    X_test, y_test = iris.data[test_index], iris.target[test_index]

    # モデルの構築
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    # モデルの評価
    print(f'Test: {lr.score(X_test, y_test)}')

Test: 0.8993541715616912
Test: 0.9261649303681716
Test: 0.9519419452318577
Test: 0.885452087053503


[1] KFold公式ドキュメント  
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

# 交差検証のバリエーション

## StratifiedKFold

### 定義

KFoldの仕方によってクラスの偏りが出ないようにクラスごとに分割したうえで、元データのクラスごとの割合を保った状態でKFoldを実施する手法

### 実装

In [82]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings('ignore')

%matplotlib inline

# データセットを読み込む
iris = load_iris()

# 交差検証の準備：n_splits=分割数、shuffle=シャッフルするかどうか、random_state:None:バラバラ、それ以外:再現性を保つ
k_fold_cross_validation = StratifiedKFold(n_splits=4, shuffle=True, random_state=1) 

# stratifiedKFoldの実施(説明変数だけでなく、目的変数を代入する必要がある！！)
for train_index, test_index in k_fold_cross_validation.split(iris.data, iris.target):

    # 選択されたインデックスをもとにデータを分割
    X_train, y_train = iris.data[train_index], iris.target[train_index]
    X_test, y_test = iris.data[test_index], iris.target[test_index]

    # モデルの構築
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    # モデルの評価
    print(f'Test: {lr.score(X_test, y_test)}')

Test: 0.9370310206615352
Test: 0.9260110826117072
Test: 0.8908901732114537
Test: 0.9470282966672622


## GroupKField

### 定義

データの中に密接な関係が存在する場合に用いられる手法。

例えば、顔の画像から人の表情を予測するモデルを作成する場合。訓練データとテストデータに同じ人の表情データがあると認識しやすくなってしまう。こうした問題を防ぐために、同一人物の表情データに対してグループのラベル付けを行い、同一の人の画像がテストデータと訓練データに分割されないようにKFoldを行う手法。

### 実装

In [110]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings('ignore')

%matplotlib inline

# データセットを読み込む
iris = load_iris()

# GroupKFoldの準備：n_splits=分割数、shuffle=シャッフルするかどうか、random_state:None:バラバラ、それ以外:再現性を保つ
k_fold_cross_validation = GroupKFold(n_splits=4) 

# GroupKFoldの実施(説明変数だけでなく、目的変数、グループラベルを代入する必要がある！！)
group = [1, 2, 3, 4, 5] * 30
for train_index, test_index in k_fold_cross_validation.split(iris.data, iris.target, group):

    # 選択されたインデックスをもとにデータを分割
    X_train, y_train = iris.data[train_index], iris.target[train_index]
    X_test, y_test = iris.data[test_index], iris.target[test_index]

    # モデルの構築
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    # モデルの評価
    print(f'Test: {lr.score(X_test, y_test)}')

Test: 0.9183206532250073
Test: 0.9097777315755673
Test: 0.9358658185205864
Test: 0.915467294366516


# 参考URL
[1]https://newtechnologylifestyle.net/%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%80%81%E3%83%87%E3%82%A3%E3%83%BC%E3%83%97%E3%83%A9%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0%E3%81%A7%E3%81%AE%E5%AD%A6%E7%BF%92%E3%83%87%E3%83%BC%E3%82%BF%E3%81%A8/

終わり